In [1]:
import re
import string
import pandas as pd

## Metin Ön İşleme

In [2]:
df = pd.read_json("fenerbahce_beko_news.json", encoding="utf-8")
df.head()

,title,text,date
0,"Final Four Üçüncülük Maçında Fenerbahçe Beko, ...",by Eurohoops Team /info@eurohoops.net\nEurohoo...,25/May/24 22:13
1,Nate Sestina: “Şampiyon Olamadık Fakat Başımız...,by Bilal Baran Yardımcı /info@eurohoops.net\nE...,25/May/24 20:39
2,Nigel Hayes-Davis: “Üçüncülük Maçı mı? Avrupa’...,by Bilal Baran Yardımcı /info@eurohoops.net\nE...,25/May/24 19:03
3,Saras Jasikevicius’tan Eurohoops’a: “Üçüncülük...,by Alex Molina /info@eurohoops.net\nEurohoops ...,25/May/24 17:49
4,EuroLeague Final Four: Tarihin En Farklı Biten...,by Utkan Şahin /info@eurohoops.net\nFinal Four...,25/May/24 08:52


In [3]:
df.shape

(9989, 3)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9989 non-null   object
 1   text    9989 non-null   object
 2   date    9989 non-null   object
dtypes: object(3)
memory usage: 234.2+ KB


In [5]:
df.iloc[0]

title    Final Four Üçüncülük Maçında Fenerbahçe Beko, ...
text     by Eurohoops Team /info@eurohoops.net\nEurohoo...
date                                       25/May/24 22:13
Name: 0, dtype: object

In [6]:
# tüm notebook için uygulanır.
# pd.set_option("display.max_colwidth", None)

# sadece satır için bu şekilde çalışır.
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[4])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   EuroLeague Final Four: Tarihin En Farklı Biten 5 Finali
text     by Utkan Şahin /info@eurohoops.net\nFinal Four, basketbolun bambaşka bir düzeyi.\nBu noktadan s

In [7]:
def text_clean(text):
    
    # 1) Başlangıçtaki "by X / email" kısmını kaldır
    text = re.sub(r"^by\s+.*?\n", " ", text, flags=re.IGNORECASE)

    # 2) Email adreslerini kaldır
    text = re.sub(r"\S+@\S+", " ", text)

    # 3) Sabit reklam / tekrar eden cümleleri temizle
    patterns = [
        r"eurohoops türkiye’nin instagram hesabını takip etmek için tıklayın!?",
        r"basketbol gündemindeki en son gelişmeleri kaçırmamak için tıklayın!?",
        r"euroleague gündemindeki son gelişmeleri kaçırmamak için tıklayın!?",
    ]
    for p in patterns:
        text = re.sub(p, " ", text, flags=re.IGNORECASE)

    # 4) Satır sonlarını boşlukla değiştir
    text = text.replace("\n", " ")

    # 5) Küçük harfe çevir
    text = text.lower()

    # 6) Harf, sayı ve noktalama dışındaki karakterleri temizle
    allowed = string.punctuation + "çğıöşüÇĞİÖŞÜ"
    text = re.sub(rf"[^a-zA-Z0-9{re.escape(allowed)}\s]", " ", text)

    # 7) Tekrarlayan noktalama işaretlerini sadeleştir
    text = re.sub(r"([!?.,:;])\1+", r"\1", text)

    # 8) Fazla boşlukları temizle
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [8]:
df["clean_text"] = df["text"].apply(text_clean)

In [9]:
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[4])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        EuroLeague Final Four: Tarihin En Farklı Biten 5 Finali
text          by Utkan Şahin /info@eurohoops.net\nFinal Four, basketbolun bambaşka bir düzeyi.\nBu 

In [10]:
with pd.option_context('display.max_colwidth', None):
    display(df.iloc[15])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Giannis Antetokounmpo Muhabir Oldu, Kardeşi Yanıtladı: “Fenerbahçe’ye Karşı Savunmayla Kazandık”
text          by Eurohoops Team /info@eurohoops.net\nEurohoo

In [11]:
# Temizlenmiş DataFrame'i JSON olarak kaydet
df.to_json("fenerbahce_beko_news_cleaned.json", orient="records", force_ascii=False, lines=True)